In [ ]:
import pandas as pd
import numpy as np

import requests

import os
import time
from datetime import datetime
import sys
import yaml
import gc


from data_loader import *
from utils import *
from nn import *


import seaborn as sns
sns.set_style("darkgrid")
plt.rcParams.update({
    'axes.facecolor': '#1e1e1e',
    'figure.facecolor': '#1e1e1e',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'white',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'axes.grid': True,
    'grid.color': 'gray'
})

pd.set_option("display.max_column",None)
print(os.getcwd())


%load_ext autoreload
%reload_ext autoreload
%autoreload 2


def auto_reload():
    %load_ext autoreload
    %reload_ext autoreload
    %autoreload 2

In [ ]:
with open("config.yaml",'r') as f:
    config = yaml.safe_load(f)

home_dir = config['HOME_DIRECTORY']
home_dir

In [ ]:
# Model File Configs:
# Every Run of the notebook is logged in to a submodel folder for it

############################################################################
# Please set output path to the project directory where it is uncompressed #
############################################################################

project_path = home_dir + "/outputs/models/"




output_path = ""

submodel_name = ""

encoding_path = ""
feature_report_path = ""
def create_submodel(model_name:str):
    author = "EJ"
    global submodel_name
    submodel_name = datetime.now().strftime("%d_%H_%M") + "_"+model_name
    global output_path
    output_path = project_path+submodel_name
    global encoding_path
    encoding_path = output_path+"/encodings/"
    global feature_report_path
    feature_report_path = output_path+"/feature_report/"
    os.mkdir(output_path)
    os.mkdir(encoding_path)
    os.mkdir(feature_report_path)


In [ ]:
create_submodel("test")

# API Football:
https://www.api-football.com

![PYTHON LOGO](https://www.api-football.com/public/img/news/archi-beta.jpg)

In [ ]:
leagues_dat = get_leagues(home_dir +"/data/Leagues/leagues.parquet")
leagues_dat[['league_id','league_name','country_name']]

# Leagues subset:

In [ ]:
# Configs
major_leagues = ["Premier League","La Liga","Serie A","Bundesliga","Eredivisie","Ligue 1"]
major_countries = ["England","Spain","Italy","Germany","Netherlands","France","Brazil"]
teams = ["Liverpool","Wolves"] # teams to pull players data of
seasons = [2022,2021,2023,2024] # seasons to pull players and teams stats of



leagues_subset = leagues_dat[leagues_dat.league_name.isin(major_leagues) & leagues_dat.country_name.isin(major_countries)] # league ID to pull from, current values: {39:premier league}, Add to dictionary as needed

In [ ]:
leagues_subset

# Read All fixtures data

In [ ]:
fixtures_dir = home_dir + "/data/Fixtures"

complete_data = pd.DataFrame()
for file in os.listdir(fixtures_dir):    
    dat = pd.read_parquet(os.path.join(fixtures_dir,file))
    complete_data = pd.concat([complete_data,dat],axis = 0)

complete_data = complete_data.reset_index()
complete_data.drop(columns = ['index'],inplace=True)

In [ ]:
# Data checks
complete_data['passes_accuracy'] = complete_data['passes_accuracy'].astype("float64")

# Targets
complete_data['outcome_num'] = pd.Categorical(complete_data.outcome).codes

complete_data['win'] = np.where(complete_data.outcome.str.lower() == 'win', 1,0)


In [ ]:
# This is the dictionary that contains all information about the features    
dat_dict = find_data_types(complete_data,config['OUTCOME_COLS'] + ['outcome_num','outcome'])
dat_dict = pd.DataFrame(list(dat_dict.items()),columns =['feature','type'])

# differentiate modeling features
non_modeling_features = config['FIXTURE_COLS'] + config['OUTCOME_COLS'] + config['MISC_COLS'] + ['outcome_num']
dat_dict['modeling_feature'] = np.where(dat_dict['feature'].isin(non_modeling_features),0,1)
dat_dict['encoded'] = 0

print(dat_dict['type'].value_counts())
dat_dict.reset_index(drop= True)
dat_dict

In [ ]:
## Encode Features
dat_dict = create_data_index(complete_data,dat_dict,'target',encoding_path)
dat_dict

In [ ]:
# quick knn clustering
from sklearn.cluster import KMeans

# who is ekitike most similar to 

stiker_stats = complete_data[['games_position','player_name'] + attacking_stat_cols].fillna(0)[complete_data[['games_position','player_name'] + attacking_stat_cols].fillna(0).games_position.isin(['F'])].groupby(['player_name']).agg(n_games = ('player_name','size'),
                                                                          avg_dribble_success = ('dribble_success_rate','mean'),
                                                                          std_dribble_success = ('dribble_success_rate','std'),
                                                                          avg_target_shot_cr = ('target_shot_conversion_perc','mean'),
                                                                          std_target_shot_cr = ('target_shot_conversion_perc','std'),
                                                                          avg_duels_cr = ('duels_won_perc','mean'),
                                                                          std_duels_cr = ('duels_won_perc','std'),
                                                                          avg_pass_accuracy = ('pass_accuracy_perc','mean'),
                                                                          std_pass_accuracy = ('pass_accuracy_perc','std')
                                                                          )


In [ ]:
stiker_stats = stiker_stats[stiker_stats.n_games > 30]
stiker_stats.shape

In [ ]:
stiker_stats

In [ ]:
stiker_stats.sort_values('avg_dribble_success',ascending=False)

In [ ]:
# among 18 players, size 3 uniform clusters
km = KMeans(10,random_state= 5)
stiker_stats['cluster_dribble'] = km.fit_predict(stiker_stats[['avg_dribble_success']])

km = KMeans(10,random_state= 5)
stiker_stats['cluster_shot_cr'] = km.fit_predict(stiker_stats[['avg_target_shot_cr']])

km = KMeans(10,random_state= 5)
stiker_stats['cluster_duels'] = km.fit_predict(stiker_stats[['avg_duels_cr']])

km = KMeans(10,random_state= 5)
stiker_stats['cluster_pass'] = km.fit_predict(stiker_stats[['avg_pass_accuracy']])

In [ ]:
# check by clusters:
stiker_stats.sort_values(["avg_target_shot_cr","cluster_shot_cr"],ascending= [0,0])[["avg_target_shot_cr","std_target_shot_cr","cluster_shot_cr"]].head(10)

In [ ]:
stiker_stats.sort_values(["avg_dribble_success","cluster_dribble"],ascending= [0,0])[["avg_dribble_success","std_dribble_success","cluster_dribble"]].head(10)

In [ ]:
stiker_stats.sort_values(["avg_duels_cr","cluster_duels"],ascending= [0,0])[["avg_duels_cr","std_duels_cr","cluster_duels"]].head(10)

In [ ]:
complete_data.columns

In [ ]:
filter_query = 'games_position.isin(["F"])'


# Stat to look at:
stat = 'target_shot_conversion_perc'

# configs 
min_appearance = 40

dribble_dat_g = complete_data[complete_data.games_position.isin(['F'])].reset_index().groupby("player_name").agg(n_apps = ("player_name","size"),stat = (stat,"median")).reset_index()
dribble_dat_g = dribble_dat_g[dribble_dat_g.n_apps >= min_appearance]
dribble_dat_g['rank'] = dribble_dat_g["stat"].fillna(0).rank(ascending= False,method = 'dense')
dribble_dat_g.sort_values("rank",inplace = True)

fig, ax = plt.subplots(figsize=(13, 8))

# Plot correctly, no comma here
sns.boxplot(
    data=dribble_dat[dribble_dat.player_name.isin(dribble_dat_g[dribble_dat_g['rank'] < 15]['player_name'])],
    x="player_name",
    y=stat,
    order=dribble_dat_g[dribble_dat_g['rank'] < 15]['player_name'],
    ax=ax,
    
)

# Now this works correctly on `ax`
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_title(f"Stat: {stat}")
plt.tight_layout()
plt.show()


In [ ]:
complete_data[['outcome_num','outcome']].value_counts()

In [ ]:
fig = plt.subplots(nrows=1, ncols = 1, figsize = (20,10))
fig = sns.heatmap(complete_data.query(filter_query)[config['ATTACK_COLS'] + ['team_goals_scored','team_non_penalty_goals_scored','team_goals_conceded']].corr(),cmap = 'coolwarm')
fig.set_xticklabels(fig.get_xticklabels(),rotation = 60)


In [ ]:
# trial multiclass model:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(complete_data[list(set(config['DEFENSE_COLS'] + config['PASSING_COLS'] + config['ATTACK_COLS'])) + ['outcome_num']].drop(columns = 'outcome_num'),complete_data[list(set(config['DEFENSE_COLS'] + config['PASSING_COLS'] + config['ATTACK_COLS'])) + ['outcome_num']][['outcome_num']],stratify=nn_dat[['outcome_num']],random_state=33)


In [ ]:
dat = NNDataFromPd(X_train.fillna(0), y_train.outcome_num, dat_dict)
train_loader = DataLoader(dat, batch_size = 128,shuffle= True)

In [ ]:
train_loader.dataset.X_numeric_tensor.shape

In [ ]:
# model params
n_features = X_train.shape[1]
n_classes = y_train.iloc[:,0].nunique()
model = MultiClassModel(n_features,n_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
epochs = 500

for epoch in range(epochs):
    
    epoch_loss = 0

    for X_numeric_batch, X_categoric_batch, y_batch in train_loader:
        
        pred = model.forward(X_numeric_batch)
        
        loss = criterion(pred,y_batch)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch: {epoch}, Loss: {epoch_loss}")

In [ ]:
model.eval()

In [ ]:
X_test['passes_accuracy'] = X_test['passes_accuracy'].astype("float64")

In [ ]:
# test sets

test_dat = NNDataFromPd(X_test,y_test,dat_dict)
test_loader = DataLoader(test_dat,batch_size= X_test.shape[0],shuffle=True)

In [ ]:
model.eval()
with torch.no_grad():
    for X_numeric_batch, X_categoric_batch, y_batch in test_loader:
        output = model(X_numeric_batch)
        pred_class = torch.argmax(output, dim = 1)

In [ ]:
# Logistic Model:
X_train, X_test, y_train, y_test = train_test_split(complete_data[complete_data.games_position == 'F'][list(set(config['DEFENSE_COLS'] + config['PASSING_COLS'] + config['ATTACK_COLS'])) + ['win']].drop(columns = 'win'),
                                                    complete_data[complete_data.games_position == 'F']['win'],
                                                    stratify=complete_data['win'],
                                                    random_state=33)

In [ ]:
train_dat = NNDataFromPd(X_train,y_train,dat_dict)
train_loader = DataLoader(train_dat,batch_size= 128,shuffle = True)

In [ ]:
n_features = X_train.shape[1]
model = LogisticNNModelComplex(n_features)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.005)

In [ ]:
epochs = 500
for epoch in range(epochs):
    epoch_loss = 0
    
    for X_numeric, X_categoric, y in train_loader:

        pred = model(X_numeric)

        loss = criterion(pred,y.unsqueeze(1))

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch},  Loss: {loss}")


In [ ]:
pred_proba.squeeze(1)

In [ ]:
from validations import *


test_dat = NNDataFromPd(X_test,y_test,dat_dict)
test_loader = DataLoader(test_dat,batch_size= X_test.shape[0],shuffle=True)

model.eval()
with torch.no_grad():
    for X_numeric_batch, X_categoric_batch, y_batch in test_loader:
        output = model(X_numeric_batch)
        pred_proba = torch.softmax(output,dim =1)
        pred_class = torch.argmax(output, dim = 1)


discrete_evaluations(y_test,pred_class,pred_proba.squeeze(1),classification_type="Binary",model_path= output_path)